# Visual odometry pipeline

### Import requirements

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

from ImageProcessor import ImageProcessor
from helperScripts.TimeKeeper import TimeKeeper

%matplotlib ipympl

### Load images and depth map

In [2]:
folderChoice = 0
path = "".join(["testImages/visualOdometryTestImages/", str(folderChoice)])

# Load set of top images
imageGlobT = sorted(glob.glob("".join([path, "/top_*", ".png"])))

# Load set of bottom images
imageGlobB = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))

# Load depth map
imageGlobD = sorted(glob.glob("".join([path, "/topDepth_*", ".png"])))

if not (len(imageGlobT)==len(imageGlobB) and \
                len(imageGlobB)==len(imageGlobD)):
    print("Images could not be matched")

print ("Selections: 0-{}".format(len(imageGlobT)-1))

Selections: 0-1


In [3]:
# Top images
imagesT = []
for imageFile in imageGlobT:
    imagesT.append(cv2.imread(imageFile))

# Bottom images
imagesB = []
for imageFile in imageGlobB:
    imagesB.append(cv2.imread(imageFile))

# Depth maps; -1 flag to load them as is
depthMaps = []
for imageFile in imageGlobD:
    depthMaps.append(cv2.imread(imageFile, -1))

print(len(imagesT))
print(len(imagesB))
print(len(depthMaps))

2
2
2


### View top camera images

In [4]:
plt.figure()
plt.suptitle("Top images")
plt.imshow(cv2.cvtColor(np.hstack(imagesT), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### View bottom camera images

In [5]:
plt.figure()
plt.suptitle("Bottom images")
plt.imshow(cv2.cvtColor(np.hstack(imagesB), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### View depth images
Depth images correspond to top camera

In [6]:
plt.figure()
plt.suptitle("Depth maps")
plt.imshow(np.hstack(depthMaps))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Depth map units and dimensions

In [7]:
print(depthMaps[0].shape)
print(depthMaps[0][int(depthMaps[0].shape[0]/2), \
                    int(depthMaps[0].shape[1]/2)], "mm")

(640, 360)
750 mm


### Instantiating TimeKeeper

In [8]:
timekeeper = TimeKeeper()

### Loading camera calibration matrix

In [9]:
imageProcessor=ImageProcessor()
imageProcessor.loadMonoCalibration()
k=imageProcessor.cameraMatrixL
print(k)

Reading from data/monoCalibration.json
Loaded mono calibration
[[592.63974229   0.         188.47568825]
 [  0.         592.98181459 312.47351841]
 [  0.           0.           1.        ]]


### Feature Extraction


In [10]:
def extractFeatures(orb, image):
    """Find keypoins and descriptors for the image"""
    keypoints = orb.detect(image, None)
    keypoints, descriptors = orb.compute(image, keypoints)
    
    return keypoints, descriptors

In [11]:
selection = 0

orb = cv2.ORB_create()

keypoints, descriptors = extractFeatures(orb, imagesT[selection])

print("Number of features detected in frame {}: {}"\
                                .format(selection, len(keypoints)))
print("Coordinates of first keypoint in frame {}: {}"\
                                .format(selection, str(keypoints[0].pt)))


Number of features detected in frame 0: 500
Coordinates of first keypoint in frame 0: (299.0, 596.0)


### Visualize features

In [12]:
def visualizeFeatures(image, keypoints, flag):
    """Visualize extracted features in image"""
    display = cv2.drawKeypoints(image, keypoints, None, flags=flag)
    plt.figure(figsize=(7, 7))
    plt.imshow(cv2.cvtColor(display, cv2.COLOR_BGR2RGB))

In [13]:
visualizeFeatures(imagesT[selection], keypoints, 4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
def extractAllFeatures(images, extractFeatures, orb):
    """Find keypoints and descriptors for each image in folder"""
    allKeypoints = []
    allDescriptors = []
    
    for image in images:
        keypoints, descriptors = extractFeatures(orb, image)
        allKeypoints.append(keypoints)
        allDescriptors.append(descriptors)
    
    return allKeypoints, allDescriptors

In [15]:
allKeypoints, allDescriptors = extractAllFeatures(imagesT, extractFeatures, orb)

print(len(allKeypoints), len(allDescriptors))

print("Number of features detected in frame {}: {}"\
                            .format(selection, len(allKeypoints[selection])))
print("Coordinates of the first keypoint in frame {}: {}\n"\
                            .format(selection, str(allKeypoints[selection][0].pt)))

2 2
Number of features detected in frame 0: 500
Coordinates of the first keypoint in frame 0: (299.0, 596.0)



### Feature matching

In [16]:
def matchFeatures(bfMatcher, descriptors0, descriptors1, bestNMatches):
    """Match features from two images"""
    match = bfMatcher.match(descriptors0, descriptors1)
    match = sorted(match, key = lambda x:x.distance)

    return match[:bestNMatches]

In [17]:
bfMatcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

bestNMatches = 50

match = matchFeatures(bfMatcher, allDescriptors[selection], \
                    allDescriptors[selection+1], bestNMatches)

print("Number of features matched in frames {} and {}: {}"\
                        .format(selection, selection+1, len(match)))

Number of features matched in frames 0 and 1: 50


In [25]:
def visualizeMatches(image0, keypoints0, image1, keypoints1, match):
    imageMatches = cv2.drawMatches(image0, keypoints0, \
                            image1, keypoints1, match, None, flags=2)

    plt.figure(figsize=(7, 7))
    plt.imshow(cv2.cvtColor(imageMatches, cv2.COLOR_BGR2RGB))

In [26]:
visualizeMatches(imagesT[selection], allKeypoints[selection], \
                imagesT[selection+1], allKeypoints[selection+1], match)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
def matchAllFeatures(allDescriptors, matchFeatures, bfMatcher, bestNMatches):
    """Match features for each subsequent image pair in the dataset"""
    matches = []

    for i in range(len(allDescriptors)-1):
        descriptor1 = allDescriptors[i]
        descriptor2 = allDescriptors[i+1]

        match = matchFeatures(bfMatcher, descriptor1, descriptor2, bestNMatches)

        matches.append(match)
    
    return matches

In [24]:
matches = matchAllFeatures(allDescriptors, matchFeatures, bfMatcher, bestNMatches)

print("Number of features matched in frames {} and {}: {}"\
            .format(selection, selection+1, len(matches[selection])))

Number of features matched in frames 0 and 1: 50


### Trajectory Estimation

In [13]:
def estimate_motion(match, keyp1, keyp2, k, depth0=None):
    rmat=np.eye(3)
    tvec=np.zeros((3,1))
    image1_points = []
    image2_points = []

    for m in match:
        train_idx = m.trainIdx
        query_idx = m.queryIdx

        p1x,p1y = keyp1[query_idx].pt 
        image1_points.append([p1x,p1y])

        p2x,p2y = keyp2[train_idx].pt 
        image2_points.append([p2x,p2y])

    E, mask = cv2.findEssentialMat(np.array(image1_points), np.array(image2_points), k) 

    retval,rmat,tvec,mask=cv2.recoverPose(E,np.array(image1_points),np.array(image2_points),k)

    return rmat,tvec,image1_points,image2_points      

In [14]:
i = 0
match = matches[i]
rmat, tvec, image1_points, image2_points = estimate_motion(match, keyp1, keyp2, k, depth0=depth0)

print("Estimated rotation:\n {0}".format(rmat))
print("Estimated translation:\n {0}".format(tvec))

Estimated rotation:
 [[ 0.99445956 -0.03551991  0.09893699]
 [ 0.04459262  0.99484699 -0.09105477]
 [-0.09519291  0.09496215  0.99091902]]
Estimated translation:
 [[-0.65096869]
 [ 0.63698372]
 [ 0.41290617]]


Camera Movement Visualization

In [15]:
def visualize_camera_movement(image1, image1_points, image2, image2_points, is_show_img_after_move=False):
    
    
    for i in range(0, len(image1_points)):
        # Coordinates of a point on t frame
        p1 = (int(image1_points[i][0]), int(image1_points[i][1]))
        # Coordinates of the same point on t+1 frame
        p2 = (int(image2_points[i][0]), int(image2_points[i][1]))

        cv2.circle(image1, p1, 5, (0, 255, 0), 1)
        cv2.arrowedLine(image1, p1, p2, (0, 255, 0), 1)
        cv2.circle(image1, p2, 5, (255, 0, 0), 1)

        if is_show_img_after_move:
            cv2.circle(image2, p2, 5, (255, 0, 0), 1)
    
    if is_show_img_after_move: 
        return imageT1
    else:
        return imageT0

In [16]:
image_move = visualize_camera_movement(imageT0, image1_points, imageT1, image2_points)
plt.figure(figsize=(7,7),dpi=100)
plt.imshow(image_move)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
image_move = visualize_camera_movement(imageT0, image1_points, imageT1, image2_points, is_show_img_after_move=True)
plt.figure(figsize=(7, 7))
plt.imshow(image_move)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Camera Trajectory Estimation

In [18]:
def estimate_trajectory(estimate_motion, matches):
    trajectory = [np.array([0, 0, 0])]
    R = np.diag([1,1,1])
    T = np.zeros([3, 1])
    RT = np.hstack([R, T])
    RT = np.vstack([RT, np.zeros([1, 4])])
    RT[-1, -1] = 1
    for i in range(len(matches)):     
        match = matches[i]

        rmat, tvec, image1_points, image2_points = estimate_motion(match, keyp1, keyp2, k)
        rt_mtx = np.hstack([rmat, tvec])
        rt_mtx = np.vstack([rt_mtx, np.zeros([1, 4])])
        rt_mtx[-1, -1] = 1

        rt_mtx_inv = np.linalg.inv(rt_mtx)
        
        RT = np.dot(RT, rt_mtx_inv)
        new_trajectory = RT[:3, 3]
        trajectory.append(new_trajectory)

    trajectory = np.array(trajectory).T   
    return trajectory 

In [19]:
trajectory = estimate_trajectory(estimate_motion, matches, kp_list, k, depth_maps=[])

i=0
print("Camera location in point {0} is: \n {1}\n".format(i, trajectory[:, [i]]))
print("Length of trajectory: {0}".format(trajectory.shape[0]))

Reading from data/monoCalibration.json
Loaded mono calibration


TypeError: estimate_motion() missing 4 required positional arguments: 'match', 'keyp1', 'keyp2', and 'k'